In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint

%matplotlib inline

In [3]:
df = pd.read_csv("Poker.csv", encoding='latin-1')
df.head()

,Player Name,Site,Hands,Net Won,VP$IP,PFR,3Bet,Postflop Agg%,W$WSF%,WTSD%,...,Raise River CBet,Squeeze,Raise Two Raisers,Call Two Raisers,vs 3Bet Fold,vs 3Bet Call,vs 3Bet Raise,vs 4Bet Fold,vs 4Bet Call,vs 4Bet Raise
0,LocSta,22,2035485,128473.89,0.249043,0.210084,0.094206,0.351073,0.477881,0.293348,...,0.113415,0.102155,0.040503,0.016301,0.604096,0.242094,0.154423,0.354854,0.284278,0.360868
1,.PARTOUCHE.,22,178311,15940.79,0.251039,0.201261,0.077332,0.314294,0.422270,0.242501,...,0.135371,0.076993,0.026232,0.010154,0.679106,0.209865,0.112322,0.481436,0.191832,0.326733
2,julien8082,22,110413,20157.79,0.277494,0.239465,0.077709,0.374164,0.475568,0.241437,...,0.190751,0.081782,0.044171,0.005646,0.656114,0.229410,0.115623,0.471349,0.173752,0.354898
3,PapiersSVP,0,103957,17549.39,0.257664,0.221630,0.109587,0.345363,0.467605,0.300238,...,0.099010,0.107238,0.051926,0.016876,0.541318,0.270994,0.188692,0.343150,0.281690,0.375160
4,NextPlease,22,99879,7326.03,0.260485,0.212327,0.082384,0.328379,0.461007,0.263755,...,0.147826,0.067406,0.023245,0.028610,0.596575,0.290834,0.114494,0.520629,0.108055,0.371316


In [6]:
pprint(list(df.columns))

['Site',
 'Hands',
 'Net Won',
 'VP$IP',
 'PFR',
 '3Bet',
 'Postflop\nAgg%',
 'W$WSF%',
 'WTSD%',
 'Won $\nat SD',
 'Flop\nCBet%',
 'Turn\nCBet%',
 'River\nCBet%',
 'Fold to\nFlop Cbet',
 'Fold to\nTurn CBet',
 'Fold to\nRiver CBet',
 'Raise\nFlop Cbet',
 'Raise\nTurn CBet',
 'Raise\nRiver CBet',
 'Squeeze',
 'Raise Two\nRaisers',
 'Call Two\nRaisers',
 'vs 3Bet\nFold',
 'vs 3Bet\nCall',
 'vs 3Bet\nRaise',
 'vs 4Bet\nFold',
 'vs 4Bet\nCall',
 'vs 4Bet\nRaise']


In [5]:
df = df[df['Hands'] > 500].drop(["Player Name"], axis=1)

## Build the target Net Won > 0

## Take care of class imbalance (more losing players than winning players)

In [17]:
df.head()

,Site,Hands,Net Won,VP$IP,PFR,3Bet,Postflop Agg%,W$WSF%,WTSD%,Won $ at SD,...,Raise River CBet,Squeeze,Raise Two Raisers,Call Two Raisers,vs 3Bet Fold,vs 3Bet Call,vs 3Bet Raise,vs 4Bet Fold,vs 4Bet Call,vs 4Bet Raise
0,22,2035485,128473.89,0.249043,0.210084,0.094206,0.351073,0.477881,0.293348,0.484962,...,0.113415,0.102155,0.040503,0.016301,0.604096,0.242094,0.154423,0.354854,0.284278,0.360868
1,22,178311,15940.79,0.251039,0.201261,0.077332,0.314294,0.422270,0.242501,0.547249,...,0.135371,0.076993,0.026232,0.010154,0.679106,0.209865,0.112322,0.481436,0.191832,0.326733
2,22,110413,20157.79,0.277494,0.239465,0.077709,0.374164,0.475568,0.241437,0.514436,...,0.190751,0.081782,0.044171,0.005646,0.656114,0.229410,0.115623,0.471349,0.173752,0.354898
3,0,103957,17549.39,0.257664,0.221630,0.109587,0.345363,0.467605,0.300238,0.495534,...,0.099010,0.107238,0.051926,0.016876,0.541318,0.270994,0.188692,0.343150,0.281690,0.375160
4,22,99879,7326.03,0.260485,0.212327,0.082384,0.328379,0.461007,0.263755,0.530951,...,0.147826,0.067406,0.023245,0.028610,0.596575,0.290834,0.114494,0.520629,0.108055,0.371316


## Running logistic regression